# Inverse kinematics of a 3dof manipulator
Consider the following manipulator

<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/3d-3dof-revolute.png?raw=true" width=500 />

The inverse kinematics can be stated as follows. Given a desired position $p$ of the tool center point at the end of the second link, determine the joint angles $\theta_1$, $\theta_2$ and $\theta_3$ that achieves this position.

Let $ f(\theta) = p$ be the forward kinematics map. Here an iterative solution is presented, solving the inverse kinematics using Newton's method for finding the solution to 
$$ f(\theta) - p = 0. $$

In each iteration of the algorithm we have an (inaccurate) solution $\theta^{(i)}$, and approximate the left hand side of the equation by a straight line 
$$ f(\theta) - p \approx f(\theta^{i}) - p + \underbrace{\frac{\partial}{\partial\theta} f(\theta)|_{\theta^{(i)}}}_{\text{Jacobian,}\, J(\theta^{(i)})} \underbrace{\left( \theta - \theta^{(i)}\right)}_{\Delta\theta}. $$
This gives
$$ f(\theta^{i}) - p + J(\theta^{(i)}) \Delta\theta = 0 $$
with solution 
$$ \Delta\theta = J^{-1}(\theta^{(i)}) \left( p - f(\theta^{i})\right). $$
This requires that the Jacobian is invertible.
So the next value in the iterative algorithm becomes
$$ \theta^{(i+1)} = \theta^{(i)} + \Delta\theta. $$
Iterate until the update is sufficiently small $|\Delta\theta| < \epsilon$, or a maximum number of iterations are reached.

See [Modern Robotics, Numerical inverse kinematics](https://youtu.be/VhUA0jf7tI8So).

In [1]:
import numpy as np
import doctest
import sympy as sy
sy.init_printing()

## Find the forward map and jacobian

In [2]:
def forward_map_and_jacobian(l1=2, l2=1):
    """
    Returns the forward map and the jacobian for the manipulator arm as callable function objects.
      
    Arguments
    ---------
    l1, l2 : float
       Length of the two links, respectively.
 
    Returns
    -------
    f : callable
       The forward kinematics map
    J : callable
       The Jacobian
    
    Tests
    ------------
    >>> f, J = forward_map_and_jacobian(l1=2, l2=1)
    >>> #1) End-effector pose at default position
    >>> wp = f(0,0,0) 
    >>> "(%0.2f, %0.2f, %0.2f)" %(wp[0], wp[1], wp[2])
    '(1.00, 0.00, 2.00)'
    >>> # 2) End-effector pose at 90 degrees in all three joints
    >>> wp = f(np.pi/2, np.pi/2, np.pi/2)
    >>> "(%0.2f, %0.2f, %0.2f)" %(wp[0], wp[1], wp[2])
    '(0.00, 1.00, 0.00)'
    >>> # 3) End-effector pose at 0 degress in first joint and 90 degress in second
    >>> wp = f(0, np.pi/2, 0)
    >>> "(%0.2f, %0.2f, %0.2f)" %(wp[0], wp[1], wp[2])
    '(2.00, 0.00, -1.00)'
    
    """
    
    th1, th2, th3 = sy.symbols('theta_1, theta_2, theta_3')
    c1 = sy.cos(th1)
    s1 = sy.sin(th1)
    c2 = sy.cos(th2)
    s2 = sy.sin(th2)
    c3 = sy.cos(th3)
    s3 = sy.sin(th3)

    # First joint, rotation about z.
    T_01 = sy.eye(4)
    T_01[:3, 0] = [c1, s1, 0] # x_1 in {0}
    T_01[:3, 1] = [-s1, c1, 0] # y_1 in {0}
    
    # Second joint, rotation about y
    T_12 = sy.eye(4)
    T_12[:3, 0] = [c2, 0, -s2] # x_2 in {1}
    T_12[:3, 2] = [s2, 0, c2] # z_2 in {1}
    
    # Link 1, origin in joint 3, directions same as {2}
    T_2L1 = sy.eye(4)
    T_2L1[:3, 3] = [0, 0, l1]
    
    # Third joint, connecting the two segments. Rotation about y_L1 = y_2 = y1
    T_L13 = sy.eye(4)
    T_L13[:3, 0] = [c3, 0, -s3] # x_2 in {1}
    T_L13[:3, 2] = [s3, 0, c3] # z_2 in {1}
    
    # Combined transform
    T_0E = T_01 @ T_12 @ T_2L1 @ T_L13
    
    # Wrist point, at tip of second segment
    wp0 =  sy.Matrix([[l2], [0], [0], [1]])
    # Wrist point in base frame, this is the forward map
    wp = sy.Matrix((T_0E @ wp0)[:3])
    f_lam = sy.lambdify((th1, th2, th3), wp, 'numpy')
    
    # Now the jacobian.
    J = wp.jacobian((th1, th2, th3))
    j_lam = sy.lambdify((th1, th2, th3), J, 'numpy')
    
    return (f_lam, j_lam)
    
    
    
    

In [3]:
doctest.run_docstring_examples(forward_map_and_jacobian, globals())

## The iterative algorithm

In [75]:
import pdb
def iterative_inverse_kinematics(wp, theta0=np.zeros(3), l1=2, l2=1, 
                                 step_size = 1, tol=1e-8, max_it = 1000):
    """
    Solves the inverse kinematics problem iteratively, using Newton's method.
    
    Arguments
    ---------
    wp  : nd array (3,)
       The wrist point in the base frame.
    theta0 : nd array (3,)
       The initial guess.
    l1, l2 : float
       The length of the segments.
    step_size : float
       The size of the step in each iteration. step_size=1 gives the Newton's method
    tol : float
       The tolerance for the norm of the angle updates
    max_it : int
       Maximum number of iterations
       
    Returns
    --------
    theta : nd array (3,)
       The joint angles.
       
    Tests
    --------
     1) Arm stretched out along the x-axis
    >>> wp = np.array([3,0,0])
    >>> thtrue = np.array([0, np.pi/2, -np.pi/2])
    >>> th = iterative_inverse_kinematics(wp, 0.5*thtrue)
    >>> # th, thtrue
    >>> np.allclose(th, thtrue, atol=1e-4)
    True
    
    2) Arm just rotated 90 degrees about first joint
    >>> wp = np.array([0,1,2])
    >>> thtrue = np.array( [np.pi/2, 0, 0] )
    >>> th = iterative_inverse_kinematics(wp, 0.8*thtrue)
    >>> # th, thtrue
    >>> np.allclose(th, thtrue, atol=1e-4)
    True
    
    3) Use the forward map for some random angles. Make sure the elbow up
    configuration is used. Make intial guess reasonably close
    >>> ths = np.random.randn(3)
    >>> th1 = ths[0]
    >>> th2 = np.clip(ths[1], 0, np.pi)
    >>> th3 = np.clip(ths[2], -np.pi/2, np.pi/2)
    >>> fk, J = forward_map_and_jacobian()
    >>> wp = fk(th1, th2, th3)
    >>> thtrue = np.array([th1, th2, th3])
    >>> th = iterative_inverse_kinematics(wp, 0.8*thtrue)
    >>> # th, thtrue
    >>> np.allclose( th, thtrue, atol=1e-4)
    True
    """
    
    f, J = forward_map_and_jacobian(l1, l2)
    
    wp = np.ravel(wp)
    
    #print(J(*theta0))
    theta_i = theta0
    dth_norm = 1e10
    it = 0
    while (dth_norm > tol) and (it < max_it):
        it += 1    
        Ji = J(*theta_i)
        #dth = np.linalg.solve(Ji, wp - np.ravel(f(*theta_i)))
        # Use pseudo-inverse instead.
        dth = np.linalg.pinv(Ji, rcond=1e-10) @ (wp - np.ravel(f(*theta_i)))
        dth_norm = np.linalg.norm(dth)
        theta_i += step_size*np.ravel(dth)
    
    if it == max_it:
        print("Warning: Reached max iterations")

    return theta_i
    
    
    

In [76]:
doctest.run_docstring_examples(iterative_inverse_kinematics, globals())